# Índice
* [Scraping](#Web-scraping)
* [Limpieza y Feature engineering](../limpieza/Limpieza-y-exploracion-de-datos-no-estructurados-con-spark.ipynb)
* [Modelado](../modelado/Modelado-y-visualizaciones.ipynb)

# Web scraping

### Objetivo
Banxico es una fuente oficial de datos que tiene muchas páginas web hechas con Flash con un bonito botón de "Descargar a CSV". **Nuestro ideal es incorporar fuentes de datos relevantes en forma automatizada.**


¿Cuántas fuentes de datos serán incorporadas?
Aquéllas que sean relevantes.

Scraping es una de las herramientas más poderosas porque nos permite "generar" nuestras propias fuentes de datos. Por ejemplo,
    * ¿Cuántas personas caminan/trabajan/viven en esta colonia?
        Podemos usar fuentes públicas + algo de redes sociales
    * ¿Mi restaurante es relevante?
        Crucemos datos de Google Places + Twitter + Foursquare

Tomaremos como referencia los datos de **Tasas de interés con rendimiento liquidable al vencimiento y de depósitos a plazo fijo , en moneda nacional, de la banca**. Esta es una de esas fuentes de datos en una página en flash. Abrir el siguiente URL en un navegador Chrome (de preferencia).

- Fuente:Banxico
    - http://www.banxico.org.mx/portal_disf/tpa.html
    
    
1. Jugar un poco con el sitio web mientras vemos el código HTML y las peticiones de red que se ejecutan ante una interacción con la página.

2. Tomamos una de estas peticiones como CURL.
    * Click derecho -> Inspeccionar.
        Aparecerá una pantalla donde podremos ver el codigo HTML + consola de javascript + monitoreo de redes
    * Click en la pestaña "Networking".
    * Aparecerán todas las peticiones hechas hasta ese momento 
![Aglo va aqui](./imgs/inspect_curl.png)
3. Probamos el resultado dentro de este notebook de la siguiente manera

In [ ]:
# Hacemos una petición fuera de python, en bash con el signo de !
! curl 'http://www.banxico.org.mx/portal_disf/Fechas_mensuales' \
    -H 'Accept-Language: en-US,en;q=0.9' \
    -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36' \
    -H 'Accept: */*' \
    -H 'Referer: http://www.banxico.org.mx/portal_disf/tpa.html' \
    -H 'X-Requested-With: ShockwaveFlash/29.0.0.171' \
    -H 'Connection: keep-alive'

¿Qué sabemos del sitio?
1. Todo sitio hecho con Flash, si tiene contenido dinámico, es probable que se generen peticiones a algún sitio web para actualizar su contenido.
2. Podemos calcular el mínimo número de Headers y parámetros para que el sitio web responda algo coherente.
3. Nos da información del tipo de server del cliente.
4. Si podemos recrear interacciones de una página web con CURL, crear un scraper con puro código python es muy factible.


Ahora que sabemos que el sitio de Banxico nos responde con CURL, usaremos Python para hacer las llamadas al sitio. Algunas herramientas son:
    * requests
        Peticiones HTTP
    * selenium (existe en java también)
        Podemos hacer interacción automatizadas a través de Chrome o Firefox
    * Beautiful Soap (bs4)
        Traductor de texto formateado con HTML a cosas que Python entiende
    * Scrapy
        Una araña tipo google que, dado un sitio web en concreto, lo puede recorrer en busca de información

In [ ]:
# Librería para peticiones
import requests

In [ ]:
# Hacemos petición a la url
url = 'http://www.banxico.org.mx/portal_disf/Recupera_tpa'
r = requests.get(url)
r.text

Una petición al URL directo regresa cosas que no tienen mucho sentido y no se parecen  a lo que vimos en Chrome. Entonces, agregaremos las variables que la petición original envía y veamos qué pasa.

In [ ]:
# Marca error. Hay que mandar variables para la búsqueda
url = 'http://www.banxico.org.mx/portal_disf/Recupera_tpa'
data = {
    'BMXC_orden':"T",
    'BMXC_plazo_ini':"30",
    'BMXC_fecha_fin':"2040/12/31",
    'BMXC_institucion':"TODAS",
    'BMXC_semaforo':"0",
    'BMXC_plazo_fin':"30",
    'BMXC_monto_ini':"10000 ",
    'BMXC_periodicidad':"H",
    'BMXC_monto_plazo':"NA",
    'BMXC_monto_fin':"10000 ",
    'BMXC_fecha_ini':"2040/12/31"
}
r = requests.get(url, params=data)


r.text

Esto es parecido a lo que obtenemos de la página web. Por completez, agregamos algunos Headers a la petición para enmascarar nuestra interacción con el sitio.


In [ ]:
# Y si además queremos que no nos cachen
url = 'http://www.banxico.org.mx/portal_disf/Recupera_tpa'
data = {
    'BMXC_orden':"T",
    'BMXC_plazo_ini':"30",
    'BMXC_fecha_fin':"2040/12/31",
    'BMXC_institucion':"TODAS",
    'BMXC_semaforo':"0",
    'BMXC_plazo_fin':"30",
    'BMXC_monto_ini':"10000 ",
    'BMXC_periodicidad':"H",
    'BMXC_monto_plazo':"NA",
    'BMXC_monto_fin':"10000 ",
    'BMXC_fecha_ini':"2040/12/31"
}

headers = {
    'Origin': 'http://www.banxico.org.mx',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'en-US,en;q=0.9',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': '*/*',
    'Referer': 'http://www.banxico.org.mx/portal_disf/tpa.swf',
    'X-Requested-With': 'ShockwaveFlash/29.0.0.171',
    'Connection': 'keep-alive'
}

In [ ]:
r = requests.get(url,params=data,headers=headers)

In [ ]:
r.text

Podemos procesar XML en Python con ayuda de la biblioteca **xml.etree**.

In [ ]:
import xml.etree.ElementTree as ET
root = ET.fromstring(r.text)

In [ ]:
root

In [ ]:
root.tag

In [ ]:
for child in root:
    for x in child:
        print(x.tag,x.text)

La interacción es fea, torpe y realmente no nos gusta. Preferimos transformar el XML a JSON y procesar a partir de este punto. Instalaremos la biblioteca **xmltodict**.

Podemos ver visualmente un XML y un JSON en la siguiente imagen.

![XML vs JSON](./imgs/json-vs-xml.png)

In [ ]:
!pip install xmltodict

In [ ]:
import xmltodict, json

In [ ]:
banxico = xmltodict.parse(r.text)

In [ ]:
banxico

In [ ]:
banxico['obj']

In [ ]:
banxico['obj']['item']

In [ ]:
for x in banxico['obj']['item']:
    if "col1" in x:
        print(x)

In [ ]:
[x for x in banxico['obj']['item'] if "col1" in x]

In [ ]:
len([x for x in banxico['obj']['item'] if "col1" in x])

In [ ]:
len(banxico['obj']['item'])

In [ ]:
banxico['obj']['registros']

In [ ]:
banxico_limpio = [x for x in banxico['obj']['item'] if "col1" in x]

In [ ]:
import csv
keys = banxico_limpio[0].keys()
keys

In [ ]:

with open('banxico.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(banxico_limpio)

In [ ]:
len(banxico_limpio)

In [ ]:
keys = {
    'col1': 'institucion',
    'col2': 'tasa_de_interes_bruta_en_porcentaje_anual',
    'col3': 'rango_de_plazo_plazo_minimo',
    'col4': 'rango_de_plazo_plazo_maximo',
    'col5': 'tipo_de_instrumento',
    'col6': 'rango_de_inversion_monto_minimo',
    'col7': 'rango_de_inversion_monto_maximo',
    'col8': 'tipo_de_tasa',
    'col9': 'fecha_de_la_informacion'
}

In [ ]:
list(map(lambda x: dict((keys[key], value) for (key, value) in x.items()), banxico_limpio))

In [ ]:
len(list(map(lambda x: dict((keys[key], value) for (key, value) in x.items()), banxico_limpio)))

In [ ]:
len(banxico_limpio)

In [ ]:
keys_csv = [
    'institucion',
    'tasa_de_interes_bruta_en_porcentaje_anual',
    'rango_de_plazo_plazo_minimo',
    'rango_de_plazo_plazo_maximo',
    'tipo_de_instrumento',
    'rango_de_inversion_monto_minimo',
    'rango_de_inversion_monto_maximo',
    'tipo_de_tasa',
    'fecha_de_la_informacion'
    ]

In [ ]:
with open('banxico_headers.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys_csv)
    dict_writer.writeheader()
    dict_writer.writerows(
        list(map(lambda x: dict((keys[key], value) for (key, value) in x.items()), banxico_limpio))
    )